In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

if os.getcwd().endswith('notebooks'):
    # Move up one level to the project root
    os.chdir('..') 

print(f"Current Working Directory: {os.getcwd()}")

Current Working Directory: /home/hsozer/Projects/DeepLearning/recipe_bot


In [4]:
from scripts.rag_builder import RecipeRAGBuilder

# Configuration
DATA_PATH = "data/processed/final_recipes.pkl"
DB_PATH = "chroma_db"

print("Libraries loaded.")

Libraries loaded.


In [6]:
# We create an instance of our builder class
builder = RecipeRAGBuilder(db_path=DB_PATH)

Skipping import of cpp extensions due to incompatible torch version 2.6.0+cu124 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# We inspect the dataframe before ingesting to make sure it's correct
df = builder.load_data(DATA_PATH)
df.head(3)

📂 Loading data from data/processed/final_recipes.pkl...
✅ Data Loaded. Shape: (226816, 22)


,name,id,minutes,tags,n_steps,steps,description,ingredients,n_ingredients,calories,...,sodium_pdv,protein_pdv,sat_fat_pdv,carbs_pdv,total_fat_g,sugar_g,sodium_mg,protein_g,sat_fat_g,carbs_g
0,easy as can be creamed turkey veggies,147023,35,"[60-minutes-or-less, time-to-make, course, mai...",7,"[in a medium sized bowl add soup and milk , mi...",if you are searching for a recipe to use that ...,"[turkey, campbell's cream of chicken soup, mil...",6,114.5,...,21.0,8.0,10.0,4.0,5.20,2.5,504.0,4.0,2.0,12.0
1,mozzarella and feta chicken,277849,30,"[30-minutes-or-less, time-to-make, course, mai...",14,"[first prepare your chicken cutlets, trim the ...",this is a really easy yummy chicken dish!,"[chicken cutlets, spaghetti sauce, mozzarella ...",8,382.6,...,44.0,51.0,74.0,3.0,26.65,14.0,1056.0,25.5,14.8,9.0
2,senegalese peanut soup,57557,25,"[30-minutes-or-less, time-to-make, course, mai...",6,"[puree chickpeas , 1 / 2 cup of broth , and pe...",from a ww cookbook. very tasty and quick. low ...,"[chickpeas, chicken broth, peanut butter, pean...",11,280.6,...,39.0,27.0,9.0,12.0,9.75,13.0,936.0,13.5,1.8,36.0


In [8]:
# Initialize the database (resetting if necessary)
builder.initialize_db(reset=True)

⚙️ Connecting to ChromaDB at 'chroma_db'...
✅ Collection 'chef_cookbook' ready.


In [9]:
# Running the ingestion
builder.ingest_batch(df, batch_size=1000)

🚀 Starting Ingestion of 226816 recipes...
   Indexed 10000 / 226816...
   Indexed 20000 / 226816...
   Indexed 30000 / 226816...
   Indexed 40000 / 226816...
   Indexed 50000 / 226816...
   Indexed 60000 / 226816...
   Indexed 70000 / 226816...
   Indexed 80000 / 226816...
   Indexed 90000 / 226816...
   Indexed 100000 / 226816...
   Indexed 110000 / 226816...
   Indexed 120000 / 226816...
   Indexed 130000 / 226816...
   Indexed 140000 / 226816...
   Indexed 150000 / 226816...
   Indexed 160000 / 226816...
   Indexed 170000 / 226816...
   Indexed 180000 / 226816...
   Indexed 190000 / 226816...
   Indexed 200000 / 226816...
   Indexed 210000 / 226816...
   Indexed 220000 / 226816...
🎉 Ingestion Complete. Total Count: 226816


In [ ]:
# Verification / Search Test

import chromadb
from chromadb.utils import embedding_functions

# Connect
client = chromadb.PersistentClient(path=DB_PATH)
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)
collection = client.get_collection(name="chef_cookbook", embedding_function=embedding_func)

# Define a Test Query
query = "High protein vegan breakfast"
print(f"🔎 Searching for: '{query}'...")

# Search
results = collection.query(
    query_texts=[query],
    n_results=3,
)

# Display Results
for i in range(len(results['documents'][0])):
    doc = results['documents'][0][i]
    meta = results['metadatas'][0][i]
    print(f"\n🏆 Result {i+1}: {meta['name']}")
    print(f"   📝 Description: {doc}")
    print(f"   💪 Protein: {meta['protein_g']}g | 🔥 Calories: {meta['calories']}")
    print(f"   🥦 Ingredients: {meta['ingredients'][:100]}...") # Show first 100 chars

🔎 Searching for: 'High protein vegan breakfast'...

🏆 Result 1: protein shake  low in fat  vegan  delicious
   📝 Description: protein shake  low in fat  vegan  delicious. i think the title says it all.

its best as a meal replacement shake. i enjoy working out in the morning before breakfast, and making this my breakfast.

soy protein is unbelievably healthy and ef
   💪 Protein: 9.0g | 🔥 Calories: 258.0
   🥦 Ingredients: ['vanilla-flavored soymilk', 'water', 'tofu', 'banana', 'peanut butter', 'honey']...

🏆 Result 2: good grains pancakes
   📝 Description: good grains pancakes. from vegetarian times.
   💪 Protein: 8.5g | 🔥 Calories: 257.0
   🥦 Ingredients: ['all-purpose flour', 'rye flour', 'whole wheat flour', 'wheat germ', 'cornmeal', 'sugar', 'baking p...

🏆 Result 3: vegan pancakes
   📝 Description: vegan pancakes. 
   💪 Protein: 11.5g | 🔥 Calories: 313.5
   🥦 Ingredients: ['flour', 'sugar', 'baking powder', 'salt', 'soymilk']...


In [ ]:
# Testing the Nutritionist Pillar (Filtering)
print("🔎 Searching for 'Vegan Breakfast' with > 20g Protein...")

results = collection.query(
    query_texts=["Vegan Breakfast"],
    n_results=3,
    where={
        "$and": [
            {"protein_g": {"$gt": 20}}, # Must have > 20g protein
            {"calories": {"$lt": 600}}  # Must be under 600 calories
        ]
    }
)

for i in range(len(results['documents'][0])):
    meta = results['metadatas'][0][i]
    print(f"\n💪 Result {i+1}: {meta['name']}")
    print(f"   📈 Protein: {meta['protein_g']}g | 🔥 Calories: {meta['calories']}")

🔎 Searching for 'Vegan Breakfast' with > 20g Protein...

💪 Result 1: vegan eggplant rollatini
   📈 Protein: 20.5g | 🔥 Calories: 266.8

💪 Result 2: overnight breakfast pie
   📈 Protein: 25.5g | 🔥 Calories: 399.2

💪 Result 3: vegan gravy
   📈 Protein: 40.5g | 🔥 Calories: 484.2


In [ ]:
# Build the Safety RAG
import pandas as pd
import json

# Load Safety Data
SAFETY_FILE = "data/knowledge/safety_rules.jsonl"
safety_data = []
with open(SAFETY_FILE, 'r') as f:
    for line in f:
        safety_data.append(json.loads(line))
safety_df = pd.DataFrame(safety_data)

# Rename to match Builder
safety_df = safety_df.rename(columns={"rule": "name", "content": "description"})
safety_df['id'] = [f"safe_{i}" for i in range(len(safety_df))]

# Ingest Safety Data
safety_builder = RecipeRAGBuilder(db_path=DB_PATH, collection_name="chef_safety")
safety_builder.initialize_db(reset=True)
safety_builder.ingest_batch(safety_df)

print("🛡️ Safety Pillar (Pillar 1) is now active!")

⚙️ Connecting to ChromaDB at 'chroma_db'...
🗑️ Deleted existing collection: chef_safety
✅ Collection 'chef_safety' ready.
🚀 Starting Ingestion of 44 items...
🎉 Ingestion Complete. Total Count: 44
🛡️ Safety Pillar (Pillar 1) is now active!


In [18]:
# Safety Search Test
query = "internal temperature for chicken"
results = safety_builder.collection.query(query_texts=[query], n_results=1)

print(f"Safety Query: {query}")
print(f"Found: {results['documents'][0][0]}")

Safety Query: internal temperature for chicken
Found: Safe Temp: Poultry. All poultry (breasts, whole bird, legs, thighs, wings, ground poultry) must be cooked to 165°F (74°C).
